In [1]:
#Import your libraries

import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import pylab as plt
import seaborn as sns
import difflib

In [2]:
flora_original = pd.read_csv('/Users/karmelealonsoaia/Desktop/ironhack_labs/PROYECTOS/project_final/data/obras_prado_flora_10000registros.csv')

flora = flora_original.copy()

# 1. Estudio de la información de iconografía en el Museo Del Prado.

# 1.1. Exploración inicial.

In [3]:
flora.head()

,uriObra,tituloObra,autores,fecha,primeros_cuatro_caracteres,Floras,Escuelas,Imagenes
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario || Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Anémona (Anemone spp.) || Gladiolo (Gladiolus ...,Italiana,https://content3.cdnprado.net/imagenes/Documen...
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla || Master of La Sisla,Hacia 1500,1495,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale || Taller de los Sarachi || ...",1570 - 1600,1570,Vid / Parra (Vitis vinifera),Italiana,https://content3.cdnprado.net/imagenes/Documen...
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Ajenuz / Arañuela (Nigella damascena) || Anémo...,Española,https://content3.cdnprado.net/imagenes/Documen...
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...


In [4]:
dim_original = flora.shape # De esta forma guardo las dimensiones del data frame original.

flora.shape

(649, 8)

In [5]:
flora.columns # Esto nos proporciona una enumeración de los nombres correspondientes a cada una de las columnas presentes en los datos.

Index(['uriObra', 'tituloObra', 'autores', 'fecha',
       'primeros_cuatro_caracteres', 'Floras', 'Escuelas', 'Imagenes'],
      dtype='object')

In [6]:
flora.info() # De este modo conzco el tipo de dato y los valores nulos que tiene cada una de las columnas.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   uriObra                     649 non-null    object
 1   tituloObra                  649 non-null    object
 2   autores                     649 non-null    object
 3   fecha                       649 non-null    object
 4   primeros_cuatro_caracteres  649 non-null    int64 
 5   Floras                      649 non-null    object
 6   Escuelas                    649 non-null    object
 7   Imagenes                    649 non-null    object
dtypes: int64(1), object(7)
memory usage: 40.7+ KB


In [7]:
# Renombrar las columnas
flora = flora.rename(columns={
    'uriObra': 'obra_uri',
    'tituloObra': 'Título',
    'autores': 'Autor',
    'fecha': 'Fecha',
    'primeros_cuatro_caracteres': 'Año',
    'Floras': 'Flora',
    'Escuelas': 'Escuela',
    'Imagenes': 'Imágenes'
})

In [49]:
columnas_a_procesar = ['Autor', 'Flora', 'Imágenes', 'Escuela']

# Reemplazar "||" solo si está entre dos palabras o fragmentos de palabras
flora[columnas_a_procesar] = flora[columnas_a_procesar].apply(lambda x: x.str.replace(r'(?<= )\|\|(?= )', ';'))

# Análisis de "Fecha" y "Año"

In [8]:
#flora['Fecha'].unique()

In [9]:
fecha_año = flora[['Fecha', 'Año']]

In [10]:
fecha_año.head()

,Fecha,Año
0,1640 - 1642,1640
1,Hacia 1500,1495
2,1570 - 1600,1570
3,Hacia 1676,1671
4,Hacia 1781,1776


In [11]:
"""
Lo que voy a hacer es, en primer lugar, dejar la columna de fecha porque solo tiene un valor numérico, lo cual
nos permite hacer operaciones con ella. También dejaré la de Fecha simplemnete para poder consultar esa información
y luego haré una nueva que contenga un rango, para luego poder crear la línea del tiempo. La denomnaré rango_fecha
""" 

'\nLo que voy a hacer es, en primer lugar, dejar la columna de fecha porque solo tiene un valor numérico, lo cual\nnos permite hacer operaciones con ella. También dejaré la de Fecha simplemnete para poder consultar esa información\ny luego haré una nueva que contenga un rango, para luego poder crear la línea del tiempo. La denomnaré rango_fecha\n'

In [12]:
"""
Para ello dejaré tal cual los valores que ya sean un rango en fecha y en caso de que no lo sean los analizaré
"""

'\nPara ello dejaré tal cual los valores que ya sean un rango en fecha y en caso de que no lo sean los analizaré\n'

In [13]:
fecha_año[fecha_año['Fecha'].str.contains('Siglo I a.C.')]

,Fecha,Año
207,Siglo I a.C.,-10


In [14]:
def transformar_fechas(fecha):
    if 'Primer cuarto del siglo XVII' in fecha:
        return '1601 - 1625'
    elif 'Siglo XVII' in fecha:
        return '1601 - 1700'
    elif 'Finales del siglo XVII - Principio del siglo XVIII' in fecha:
        return '1685 - 1715'
    elif 'Finales del siglo III a.C. - Primera mitad del siglo II a.C.' in fecha:
        return '-30 - -25'
    elif 'Siglo XVI' in fecha:
        return '1501 - 1600'
    elif 'Segunda mitad del siglo XVI' in fecha:
        return '1551 - 1600'
    elif 'Principio del siglo XX' in fecha:
        return '1901 - 1915'
    elif 'Último tercio del siglo XVIII' in fecha:
        return '1769 - 1800'
    elif 'Finales del siglo XVI - Primer tercio del siglo XVII' in fecha:
        return '1585 - 1630'
    elif 'Primera mitad del siglo XVII' in fecha:
        return '1601 - 1650'
    elif 'Segunda mitad del siglo XVII' in fecha:
        return '1651 - 1700'
    elif 'Finales del siglo XVII' in fecha:
        return '1685 - 1700'
    elif 'Siglo XVIII' in fecha:
        return '1701 - 1800'
    elif 'Finales del siglo XVIII' in fecha:
        return '1785 - 1800'
    elif 'Siglos XVI - XVII' in fecha:
        return '1501 - 1700'
    elif 'Segunda mitad del siglo XVII - Primer cuarto del siglo XVIII' in fecha:
        return '1651 - 1725'
    elif 'Siglo I a.C.' in fecha:
        return '-10'
    elif 'Finales del siglo XVI' in fecha:
        return '1685 - 1700'
    elif 'Primer tercio del siglo XVII' in fecha:
        return '1601 - 1630'
    elif 'Siglo III a.C.' in fecha:
        return '-30'
    elif 'Primer tercio del siglo XVI' in fecha:
        return '1501 - 1530'
    elif 'Segundo tercio del siglo XVII' in fecha:
        return '1634 - 1870'
    elif 'Siglo XII' in fecha:
        return '1101 - 1199'
    elif 'Último cuarto del siglo XVIII - Principio del siglo XIX' in fecha:
        return '1776 - 1815'
    elif 'Último cuarto del siglo XVII' in fecha:
        return '1676 - 1700'
    elif 'Siglos IV a.C. - III a.C.' in fecha:
        return '-40 - -30'
    elif 'Mediados del siglo XVII' in fecha:
        return '1640 - 1660'
    elif 'Siglo XIX' in fecha:
        return '1801 - 1900'
    else:
        return fecha

fecha_año['rango_fecha'] = fecha_año['Fecha'].apply(transformar_fechas)

In [15]:
fecha_año.head()

,Fecha,Año,rango_fecha
0,1640 - 1642,1640,1640 - 1642
1,Hacia 1500,1495,Hacia 1500
2,1570 - 1600,1570,1570 - 1600
3,Hacia 1676,1671,Hacia 1676
4,Hacia 1781,1776,Hacia 1781


In [16]:
#fecha_año['rango_fecha'].unique()

In [17]:
flora['rango_fecha'] = fecha_año['rango_fecha']

In [18]:
flora.head()

,obra_uri,Título,Autor,Fecha,Año,Flora,Escuela,Imágenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,Floreros,"Nuzzi, Mario || Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Anémona (Anemone spp.) || Gladiolo (Gladiolus ...,Italiana,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,La Anunciación,Maestro de la Sisla || Master of La Sisla,Hacia 1500,1495,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1500
2,http://museodelprado.es/items/E22_Man-Made_Obj...,Copa de la Vendimia,"Fontana, Annibale || Taller de los Sarachi || ...",1570 - 1600,1570,Vid / Parra (Vitis vinifera),Italiana,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,Florero,"Pérez, Bartolomé",Hacia 1676,1671,Ajenuz / Arañuela (Nigella damascena) || Anémo...,Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1676
4,http://museodelprado.es/items/E22_Man-Made_Obj...,La Inmaculada Concepción,"Maella, Mariano Salvador",Hacia 1781,1776,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1781


# Análisis "Autor"

In [21]:
flora['Título'] = flora['Título'].apply(lambda x: f'"{x}"')

In [22]:
flora.head()

,obra_uri,Título,Autor,Fecha,Año,Flora,Escuela,Imágenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Floreros""","Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)",1640 - 1642,1640,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,Italiana,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Anunciación""",Maestro de la Sisla ; Master of La Sisla,Hacia 1500,1495,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1500
2,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi ; Wo...",1570 - 1600,1570,Vid / Parra (Vitis vinifera),Italiana,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1676
4,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1781


In [23]:
flora['Autor'].unique()

array(['Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)',
       'Maestro de la Sisla ; Master of La Sisla',
       'Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi',
       'Pérez, Bartolomé', 'Maella, Mariano Salvador',
       "Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
       'Martí Garcés de Marcilla, José', 'Haes, Carlos de',
       'Tobar, Alonso Miguel de',
       'Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan',
       'Camprobín, Pedro de', 'Rincón, Fernando del', 'Hiepes, Tomás',
       'Bertodano, Luis de',
       'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid',
       'Vauquer, Jacques', 'Coosemans, Alexander', 'Leoni, Leone',
       'J. Laurent y Cía', 'Arellano, Juan de',
       'Galleria dei Lavori, Florencia',
       'Belvedere, Andrea ; Solimena, Francesco', 'Anonymous ; Anónimo',
       'Malaine, Joseph-Laurent', 'Dyck, Anthony van ; Dyck, Anton van

In [24]:
nombres_unicos = flora['Autor'].unique()
nombres_unicos = list(nombres_unicos)
nombres_unicos

['Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)',
 'Maestro de la Sisla ; Master of La Sisla',
 'Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi',
 'Pérez, Bartolomé',
 'Maella, Mariano Salvador',
 "Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
 'Martí Garcés de Marcilla, José',
 'Haes, Carlos de',
 'Tobar, Alonso Miguel de',
 'Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan',
 'Camprobín, Pedro de',
 'Rincón, Fernando del',
 'Hiepes, Tomás',
 'Bertodano, Luis de',
 'Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid',
 'Vauquer, Jacques',
 'Coosemans, Alexander',
 'Leoni, Leone',
 'J. Laurent y Cía',
 'Arellano, Juan de',
 'Galleria dei Lavori, Florencia',
 'Belvedere, Andrea ; Solimena, Francesco',
 'Anonymous ; Anónimo',
 'Malaine, Joseph-Laurent',
 'Dyck, Anthony van ; Dyck, Anton van',
 'Puebla y Tolín, Dióscoro Teófilo',
 'Maestro de la Santa Sangre',
 'David, Gérard'

In [25]:
for e in nombres_unicos:
    print(e)

Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)
Maestro de la Sisla ; Master of La Sisla
Fontana, Annibale ; Taller de los Sarachi ; Workshop of the Sarachi
Pérez, Bartolomé
Maella, Mariano Salvador
Brueghel el Viejo, Jan ; Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)
Martí Garcés de Marcilla, José
Haes, Carlos de
Tobar, Alonso Miguel de
Balen, Hendrick van ; Brueghel el Joven, Jan ; Brueghel the Younger, Jan
Camprobín, Pedro de
Rincón, Fernando del
Hiepes, Tomás
Bertodano, Luis de
Guglielmi, Augusto ; Madrazo y Agudo, José de ; Real Establecimiento Litográfico de Madrid
Vauquer, Jacques
Coosemans, Alexander
Leoni, Leone
J. Laurent y Cía
Arellano, Juan de
Galleria dei Lavori, Florencia
Belvedere, Andrea ; Solimena, Francesco
Anonymous ; Anónimo
Malaine, Joseph-Laurent
Dyck, Anthony van ; Dyck, Anton van
Puebla y Tolín, Dióscoro Teófilo
Maestro de la Santa Sangre
David, Gérard
Titian (Tiziano Vecellio) ; Tiziano, Vecellio di Gregorio
Pulido Fernández, Ramón
Pereda y Salgado, Antonio de

In [26]:
nombres_unicos[0]

'Nuzzi, Mario ; Nuzzi, Mario (Mario dei Fiori)'

In [27]:
descomposicion_de_materia = nombres_unicos[0].split(";")
descomposicion_de_materia

['Nuzzi, Mario ', ' Nuzzi, Mario (Mario dei Fiori)']

In [28]:
for i in descomposicion_de_materia:
    print(i)

Nuzzi, Mario 
 Nuzzi, Mario (Mario dei Fiori)


In [29]:
#%pip install fuzzywuzzy

In [30]:
from fuzzywuzzy import fuzz

In [31]:
def choose_author(names):
    name_list = [name.strip() for name in names.split(';')]
    
    # Calcula la similitud entre los conjuntos de palabras
    similarity_scores = []
    for i in range(len(name_list)):
        for j in range(i + 1, len(name_list)):
            score = fuzz.token_set_ratio(name_list[i], name_list[j])
            similarity_scores.append((i, j, score))
    
    # Ordena la lista de similitudes de mayor a menor
    similarity_scores.sort(key=lambda x: x[2], reverse=True)
    
    # Devuelve el nombre con mayor similitud
    if similarity_scores and similarity_scores[0][2] > 80:  # Puedes ajustar el umbral según tus necesidades
        return name_list[similarity_scores[0][0]]
    else:
        return names

In [35]:
flora['Autor'] = flora['Autor'].apply(choose_author)


In [36]:
terminos_a_eliminar = [
    'Workshop of the Sarachi',
    "Brueghel the Elder, Jan (Jan 'Velvet' Brueghel)",
    'Brueghel the Younger, Jan',
    'Anonymous',
    'Workshop of the Miseroni',
    'Bosch, Hieronymus',
    'Master of the Virgin of the Catholic Monarchs',
    'Veronés, Paolo'
]



In [37]:
for termino in terminos_a_eliminar:
    flora['Autor'] = flora['Autor'].apply(lambda x: x.replace(termino, '') if termino in x else x)

# Eliminar espacios duplicados resultantes de las sustituciones
flora['Autor'] = flora['Autor'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Si deseas eliminar las filas donde la columna 'Autor' está vacía después de la sustitución
flora = flora[flora['Autor'] != '']

# Asegurarte de que los cambios se apliquen al DataFrame original
flora.reset_index(drop=True, inplace=True)

In [39]:
def eliminar_punto_y_coma(celda):
    # Elimina el punto y coma al comienzo o al final de la cadena
    celda = celda.lstrip(';').rstrip(';')
    return celda

# Aplica la función a la columna 'Autor'
flora['Autor'] = flora['Autor'].apply(eliminar_punto_y_coma)

In [42]:
def eliminar_espacios_en_blanco(celda):
    # Elimina los espacios en blanco al comienzo o al final de la cadena
    celda = celda.lstrip().rstrip()
    return celda

# Aplica la función a la columna 'Autor'
flora['Autor'] = flora['Autor'].apply(eliminar_espacios_en_blanco)

In [45]:
#flora['Autor'].unique()

In [44]:
flora.head()

,obra_uri,Título,Autor,Fecha,Año,Flora,Escuela,Imágenes,rango_fecha
0,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Floreros""","Nuzzi, Mario",1640 - 1642,1640,Anémona (Anemone spp.) ; Gladiolo (Gladiolus s...,Italiana,https://content3.cdnprado.net/imagenes/Documen...,1640 - 1642
1,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Anunciación""",Maestro de la Sisla,Hacia 1500,1495,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1500
2,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Copa de la Vendimia""","Fontana, Annibale ; Taller de los Sarachi",1570 - 1600,1570,Vid / Parra (Vitis vinifera),Italiana,https://content3.cdnprado.net/imagenes/Documen...,1570 - 1600
3,http://museodelprado.es/items/E22_Man-Made_Obj...,"""Florero""","Pérez, Bartolomé",Hacia 1676,1671,Ajenuz / Arañuela (Nigella damascena) ; Anémon...,Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1676
4,http://museodelprado.es/items/E22_Man-Made_Obj...,"""La Inmaculada Concepción""","Maella, Mariano Salvador",Hacia 1781,1776,Azucena (Lilium candidum),Española,https://content3.cdnprado.net/imagenes/Documen...,Hacia 1781


In [47]:
#flora['Flora'].unique()

In [ ]:
Análisis "Escuela"

In [50]:
flora['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Italiana ; Veneciana',
       'Castellana ; Española', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Española ; Madrileña', 'Italiana ; Napolitana',
       'Arte Clásico ; Roma', 'Boloñesa ; Italiana', 'Gala', 'Púnica',
       'Hispano-flamenca', 'Española ; Valenciana',
       'Florentina ; Italiana', 'Inglesa', 'Sueca',
       'Castellana ; Española ; Toledana',
       'Francesa ; Italiana ; Milanesa', 'Portuguesa'], dtype=object)

In [53]:
def modificar_escuela(escuela):
    if 'Italiana ; Veneciana' in escuela:
        return 'Italiana'
    elif 'Castellana ; Española' in escuela:
        return 'Española'
    elif 'Española ; Madrileña' in escuela:
        return 'Española'
    elif 'Italiana ; Napolitana' in escuela:
        return 'Italiana'
    elif 'Arte Clásico ; Roma' in escuela:
        return 'Italiana'
    elif 'Boloñesa ; Italiana' in escuela:
        return 'Italiana'
    elif 'Española ; Valenciana' in escuela:
        return 'Española'
    elif 'Florentina ; Italiana' in escuela:
        return 'Italiana'
    elif 'Castellana ; Española ; Toledana' in escuela:
        return 'Española'
    elif 'Francesa ; Italiana ; Milanesa' in escuela:
        return 'Francesa ; Italiana'
    else:
        return escuela

# Aplica la función a la columna 'Escuela'
flora['Escuela'] = flora['Escuela'].apply(modificar_escuela)

In [54]:
flora['Escuela'].unique()

array(['Italiana', 'Española', 'Flamenca', 'Francesa',
       'Hispano-cartaginesa', 'Francesa ; Italiana', 'Holandesa',
       'Alemana', 'Gala', 'Púnica', 'Hispano-flamenca', 'Inglesa',
       'Sueca', 'Portuguesa'], dtype=object)

In [ ]:
Si pone 'Italiana ; Veneciana' dejar 'Italiana'
'Castellana ; Española' dejat Española'
Española ; Madrileña dejar española
Italiana ; Napolitana dejar italiana
Arte Clásico ; Roma dejar italiana
Boloñesa ; Italiana dejar Italiana
Florentina ; Italiana dejar Italiana
Castellana ; Española ; Toledana dejar Española
Francesa ; Italiana ; Milanesa dejar Francesa ; Italiana